In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

print("All libraries imported. Ready to build!")

All libraries imported. Ready to build!


In [ ]:
from sklearn.datasets import fetch_kddcup99
import pandas as pd

print("Fetching built-in KDD '99 dataset...")

# Load the 10% version of the dataset
# as_frame=True loads it as a pandas DataFrame, which is perfect for us
kdd_data = fetch_kddcup99(percent10=True, as_frame=True)

# The data is in 'kdd_data.frame'
df = kdd_data.frame

# The 'target' column has the attack types
df['attack_type'] = kdd_data.target

# Drop the old 'target' column if it exists (it's redundant)
if 'target' in df.columns:
    df = df.drop('target', axis=1)

print("✅ Dataset loaded successfully!")
print(f"Total connections in dataset: {len(df)}")

# --- Data Cleaning ---
# The KDD dataset has 3 text columns that we need to convert to numbers
# These are 'protocol_type', 'service', and 'flag'
print("Cleaning and preparing data...")

# We use LabelEncoder for this
from sklearn.preprocessing import LabelEncoder

# Keep track of encoders in a dictionary
encoders = {}

for col_name in ['protocol_type', 'service', 'flag']:
    le = LabelEncoder()
    # Convert column to string, then encode
    df[col_name] = le.fit_transform(df[col_name].astype(str))
    encoders[col_name] = le # Save the encoder

print("✅ Data cleaned and prepared!")
print("--- Sample of Prepared Data ---")
print(df.head())

Fetching built-in KDD '99 dataset...
✅ Dataset loaded successfully!
Total connections in dataset: 494021
Cleaning and preparing data...
✅ Data cleaned and prepared!
--- Sample of Prepared Data ---
  duration  protocol_type  service  flag src_bytes dst_bytes land  \
0        0              1       22     9       181      5450    0   
1        0              1       22     9       239       486    0   
2        0              1       22     9       235      1337    0   
3        0              1       22     9       219      1337    0   
4        0              1       22     9       217      2032    0   

  wrong_fragment urgent hot  ... dst_host_same_srv_rate  \
0              0      0   0  ...                    1.0   
1              0      0   0  ...                    1.0   
2              0      0   0  ...                    1.0   
3              0      0   0  ...                    1.0   
4              0      0   0  ...                    1.0   

  dst_host_diff_srv_rate dst_host

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# --- 1. Prepare Data for AI ---

# 'y' is the *answer* we want the AI to learn (the attack_type)
# We need to encode the attack_type labels (e.g., 'normal.', 'smurf.') into numbers
le_attack = LabelEncoder()
y = le_attack.fit_transform(df['attack_type'])

# 'X' is all the data *about* the traffic (the features)
# We drop the 'attack_type' and 'labels' columns because they are either the target or redundant.
X = df.drop(['attack_type', 'labels'], axis=1)

# Split data: 80% to train, 20% to test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 2. Train the AI Brain ---
print("Training AI Analytics model... (This may take a minute)")

# Create the AI model
ai_brain = RandomForestClassifier(n_estimators=50, random_state=42)

# Train it on the 80% of data
ai_brain.fit(X_train, y_train)

print("✅ AI Brain is trained!")

# --- 3. Test Accuracy ---
predictions = ai_brain.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"AI Brain Accuracy: {accuracy * 100:.2f}%")

Training AI Analytics model... (This may take a minute)
✅ AI Brain is trained!
AI Brain Accuracy: 99.97%


In [ ]:
# This dictionary maps the specific attacks to the 4 main categories
# This is a standard practice for this dataset
attack_map = {
    'normal.': 'Normal',

    # Denial of Service (DOS)
    'back.': 'DOS', 'land.': 'DOS', 'neptune.': 'DOS', 'pod.': 'DOS',
    'smurf.': 'DOS', 'teardrop.': 'DOS', 'mailbomb.': 'DOS', 'processtable.': 'DOS',
    'udpstorm.': 'DOS', 'apache2.': 'DOS', 'worm.': 'DOS',

    # Probing
    'ipsweep.': 'Probe', 'nmap.': 'Probe', 'portsweep.': 'Probe',
    'satan.': 'Probe', 'mscan.': 'Probe', 'saint.': 'Probe',

    # Remote to Local (R2L)
    'ftp_write.': 'R2L', 'guess_passwd.': 'R2L', 'imap.': 'R2L', 'multihop.': 'R2L',
    'phf.': 'R2L', 'spy.': 'R2L', 'warezclient.': 'R2L', 'warezmaster.': 'R2L',
    'sendmail.': 'R2L', 'named.': 'R2L', 'snmpgetattack.': 'R2L', 'snmpguess.': 'R2L',
    'xlock.': 'R2L', 'xsnoop.': 'R2L', 'httptunnel.': 'R2L',

    # User to Root (U2R)
    'buffer_overflow.': 'U2R', 'loadmodule.': 'U2R', 'perl.': 'U2R', 'rootkit.': 'U2R',
    'sqlattack.': 'U2R', 'xterm.': 'U2R', 'ps.': 'U2R'
}


def automated_response_engine(attack_prediction_encoded):
    # Convert the AI's number prediction (e.g., 1, 5, 20)
    # back to a bytes object (e.g., b'normal.', b'smurf.')
    attack_label_bytes = le_attack.inverse_transform([attack_prediction_encoded])[0]

    # --- 💡 THIS IS THE FIX 💡 ---
    # Decode the bytes object (b'normal.') into a string ('normal.')
    attack_label = attack_label_bytes.decode('utf-8')
    # --- END OF FIX ---

    # Use our map to find the main category (e.g., 'smurf.' -> 'DOS')
    # Now this will work, because attack_label is a string
    attack_category = attack_map.get(attack_label, 'Unknown')

    print(f"--- AURA ENGINE ---")
    print(f"ANALYTICS: AI detected '{attack_label}' (Category: {attack_category})")

    # Define the automated security policies based on the CATEGORY
    if attack_category == 'Normal':
        print("RESPONSE: ✅ Allow traffic. (Risk: Low)")

    elif attack_category == 'Probe':
        print("RESPONSE: ⚠️ Log & Monitor. (Risk: Medium)")
        print("-> ACTION: Flagging source IP for surveillance.")

    elif attack_category == 'DOS':
        print("RESPONSE: ⛔ BLOCK IP & Reroute. (Risk: High)")
        print("-> ACTION: Rerouting traffic to scrubbing center.")

    elif attack_category in ['R2L', 'U2R']:
        print("RESPONSE: 🛑 QUARANTINE. (Risk: Critical)")
        print("-> ACTION: Isolating compromised host and locking user account.")

    else:
        print(f"RESPONSE: ❓ Unknown type '{attack_label}'. Flag for human operator.")

    print("-------------------")


# --- Run 5 Simulations ---
print("\nRunning 5 real-time simulations...\n")

# Get 5 random samples from our test data
sample_indices = X_test.sample(5).index
sample_data = X_test.loc[sample_indices]

for i in range(5):
    # Get one piece of 'new' traffic
    new_traffic = sample_data.iloc[[i]]

    # Let the AI Brain predict it
    prediction_encoded = ai_brain.predict(new_traffic)

    # Feed the prediction to the AURA Engine
    automated_response_engine(prediction_encoded[0])


Running 5 real-time simulations...

--- AURA ENGINE ---
ANALYTICS: AI detected 'neptune.' (Category: DOS)
RESPONSE: ⛔ BLOCK IP & Reroute. (Risk: High)
-> ACTION: Rerouting traffic to scrubbing center.
-------------------
--- AURA ENGINE ---
ANALYTICS: AI detected 'normal.' (Category: Normal)
RESPONSE: ✅ Allow traffic. (Risk: Low)
-------------------
--- AURA ENGINE ---
ANALYTICS: AI detected 'normal.' (Category: Normal)
RESPONSE: ✅ Allow traffic. (Risk: Low)
-------------------
--- AURA ENGINE ---
ANALYTICS: AI detected 'normal.' (Category: Normal)
RESPONSE: ✅ Allow traffic. (Risk: Low)
-------------------
--- AURA ENGINE ---
ANALYTICS: AI detected 'smurf.' (Category: DOS)
RESPONSE: ⛔ BLOCK IP & Reroute. (Risk: High)
-> ACTION: Rerouting traffic to scrubbing center.
-------------------
